<a href="https://colab.research.google.com/github/gajeshladhar/mapminer/blob/master/tests/forest-mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>🌍 <strong>MapMiner</strong> </h1>

<h2 style="margin: 10px 0; font-size: 24px; color: #38bdf8;">
    🌲 Forest Mapping: Importance of Temporal Component
</h2>
<hr>
<p style="margin: 5px 0; font-size: 16px; line-height: 1.5; color: #e2e8f0;">
    Forest ecosystems are dynamic and constantly changing. Mapping forests accurately requires acknowledging the critical role of the <strong>time dimension</strong> in spatial datasets, often overlooked in traditional Computer Vision approaches.
</p>
<p style="margin: 5px 0; font-size: 16px; line-height: 1.5; color: #e2e8f0;">
    By adopting a <strong>spatio-temporal perspective</strong>, we can better address challenges in <strong>Earth Observation</strong>, such as tracking seasonal changes, forest degradation, and growth patterns. This approach not only improves model accuracy but also enables actionable insights for sustainable decision-making.
</p>
<p style="margin: 5px 0; font-size: 16px; line-height: 1.5; color: #e2e8f0;">
    Join us as we explore how tools like <strong>Numba</strong> and <strong>Dask</strong> revolutionize Earth Observation workflows, making them faster, scalable, and efficient to meet the demands of dynamic datasets.
</p>
#SpaceTech 🌌 #EarthObservation 🛰️ #Sustainability 🌱 #SpatioTemporal 🌏


<br><hr><h2 style="color: #16a085; font-family: 'Segoe UI', sans-serif; text-align: center; margin-top: 30px; margin-bottom: 20px;">
    🏙️🌳 Earth Observation (🌐) with Numba & Dask : Accelerating NDVI-Based Forest Detection
</h2>
<div style="text-align: center; margin-bottom: 20px;">
    <img src="https://storage.googleapis.com/p-oaf-ibe-back-00e-strapi-uploads/r49193_9_forest_and_environment_banner_10b359abd5/r49193_9_forest_and_environment_banner_10b359abd5.jpg"
         alt="Forest", height="300", width="800",
         style="border-radius: 10px;box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.1);">
</div>

<p style="color: #4a4a4a; font-size: 16px; line-height: 1.8; font-family: 'Segoe UI', sans-serif; text-align: justify;">
    <b>Problem Statement</b> : Detecting accurate forest masks over vast geographies requires the integration of temporal NDVI data spanning an entire year. The goal is to classify forests with a precision that rivals modern deep learning models, but with a fraction of the computational overhead.
    <strong>Numba</strong> supercharges this process by enabling high-performance, parallelized operations directly on raster datasets. Using lightweight just-in-time (JIT) compilation, we can efficiently process terabytes of satellite imagery to extract precise, pixel-level forest masks in near real-time. <br><br>
    <em>This approach not only democratizes access to advanced EO analytics but also sets a benchmark for harnessing Pythonic speed in geospatial data processing.</em>
</p>


In [1]:
!pip3 install rioxarray hvplot planetary_computer pystac_client odc-stac shapely cartopy geoviews

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 73.7 MB/s eta 0:00:00
  Attempting uninstall: bokeh
    Found existing installation: bokeh 3.6.2
    Uninstalling bokeh-3.6.2:
      Successfully uninstalled bokeh-3.6.2


In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray

import gc
import copy
import hvplot.xarray
import hvplot.pandas

import planetary_computer
from pystac_client import Client
from odc.stac import load
import odc.geo
from shapely import Point,Polygon

import numba
from numba import njit,prange

from IPython.display import clear_output
from dask.diagnostics import ProgressBar
from tqdm.notebook import tqdm
planetary_computer.settings.set_subscription_key("1d7ae9ea9d3843749757036a903ddb6c")

<h4 style="color: #2980b9; font-family: 'Segoe UI', sans-serif; margin-top: 20px; margin-bottom: 10px; display: flex; align-items: center;">
    🔧🚀 Setting Up: Loading Project Essential Modules and Helpers 📦📊
</h4>


In [4]:
@njit(parallel=True)
def max_rolling(x,window):
    pad_arr = np.zeros(shape=x.shape)+np.nan
    for i in prange(x.shape[0]):
        for j in prange(x.shape[1]-window+1):
            pad_arr[i,j+window-1] = np.max(x[i,j:j+window])
    return pad_arr

@njit(parallel=True)
def mean_rolling(x,window):
    pad_arr = np.zeros(shape=x.shape)+np.nan
    for i in prange(x.shape[0]):
        for j in prange(x.shape[1]-window+1):
            pad_arr[i,j+window-1] = np.mean(x[i,j:j+window])
    return pad_arr

@njit(parallel=True)
def interpolate_nan_along_axis(arr, axis=1):
    # Function to interpolate NaN values along a single axis
    def interpolate_axis(a):
        non_nan_indices = np.arange(len(a))[~np.isnan(a)]
        interp_func = np.interp(np.arange(len(a)), non_nan_indices, a[non_nan_indices])
        a[np.isnan(a)] = interp_func[np.isnan(a)]
        return a

    # Apply interpolation along the specified axis
    if axis == 0:
        for i in prange(arr.shape[1]):
            arr[:, i] = interpolate_axis(arr[:, i])
    elif axis == 1:
        for i in prange(arr.shape[0]):
            arr[i, :] = interpolate_axis(arr[i, :])
    return arr

@njit(parallel=True, cache=True)
def median_along_axis(arr):
    rows, cols = arr.shape
    result = np.empty(rows, dtype=np.float32)

    for i in prange(rows):
        valid_values = []
        for j in range(cols):
            if not np.isnan(arr[i, j]):  # Exclude NaN values
                valid_values.append(arr[i, j])

        valid_values = np.array(valid_values, dtype=np.float32)
        valid_values.sort()  # Sort valid (non-NaN) values
        n = valid_values.size

        if n == 0:  # Handle case where all values are NaN
            result[i] = np.nan
        else:
            mid = n // 2
            if n % 2 == 0:
                result[i] = (valid_values[mid - 1] + valid_values[mid]) / 2
            else:
                result[i] = valid_values[mid]

    return result

<br><h4 style="color: #27ae60; font-family: 'Segoe UI', sans-serif; margin-top: 20px; margin-bottom: 10px; display: flex; align-items: center;">
    🌿✨ Defining Maximum Green Days: A Numba-Optimized Algorithm for Forest Mask Extraction 🚀🌲
</h4>


In [5]:
@njit(parallel=True,cache=False)
def get_max_consecutive_green_days(x,min_thr=0.25,max_roll_window=2,mean_roll_window=5):
    """
    Compute Maximum Consecutive Green Days in NDVI Time Series.

    Processes a 2D NDVI time series (`x`) to calculate the maximum number of
    consecutive "green" days (NDVI > 0.25) per row after applying interpolation,
    thresholding, and smoothing.

    Parameters:
    -----------
    x : ndarray
        2D array of NDVI values (rows: spatial units, columns: time steps).
    min_thr : float, default=0.25
        Minimum NDVI threshold; values below are adjusted to min_thr.
    max_roll_window : int, default=2
        Window size for maximum rolling operation.
    mean_roll_window : int, default=5
        Window size for mean rolling operation.
    """
    xt = interpolate_nan_along_axis(x)
    for i in prange(len(x)):
        if (xt[i]>0.20).sum()<=3:
            x[i] = 0
    x = np.where(x<=min_thr,min_thr,x)
    x = np.where(np.isnan(x),0,x)
    x = max_rolling(x,max_roll_window)
    x = np.where(x==0,np.nan,x)
    x = interpolate_nan_along_axis(x)
    x = mean_rolling(x,mean_roll_window)
    x = interpolate_nan_along_axis(x[:,::-1])
    left, center, right = median_along_axis(x[:,10:19]), median_along_axis(x[:,19:24]), median_along_axis(x[:,24:30])
    x = (x[:,::-1]>0.25).astype("uint8")
    n_consecutives = np.zeros(shape=(x.shape[0]))
    for n in prange(len(n_consecutives)):
        if (center[n]<=left[n]) and (center[n]<=right[n]) and (center[n]<=0.32):
            n_consecutives[n] = 0
        else :
            arr = x[n]
            max_count = 0
            current_count = 0
            for i in arr:
                if i == 1:
                    current_count += 1
                    max_count = max(max_count, current_count)
                else:
                    current_count = 0
            n_consecutives[n] = max_count

    return n_consecutives

In [7]:
def get_max_consecutive_green_days_xarray(ds):
    out_coords = {'y':ds.y.values,'x':ds.x.values,'time':[ds.time.values[-1]]}
    for key in ds.coords:
        if key not in out_coords:
            out_coords[key] = ds.coords[key]

    dims =  copy.deepcopy(ds.dims)
    out_coords = copy.deepcopy(out_coords)
    shape = copy.deepcopy(ds.data.shape)
    ci = get_max_consecutive_green_days(ds.data.reshape(-1,shape[2]),min_thr=0.123,max_roll_window=3,mean_roll_window=3)
    del ds
    gc.collect()
    return  xr.DataArray(
                data=ci.reshape(shape[0],shape[1],1),
                dims =dims,
                coords=out_coords)

In [8]:
%%time
# Compilation.....
_ = get_max_consecutive_green_days(np.random.randint(0,2,size=(200,37)))

CPU times: user 1min 3s, sys: 319 ms, total: 1min 4s
Wall time: 1min 6s


<br><h4 style="color: #8e44ad; font-family: 'Segoe UI', sans-serif; margin-top: 20px; margin-bottom: 10px; display: flex; align-items: center;">
    🌐🔍 Initializing STAC Clients: Element 84 and Microsoft Planetary Computers 🚀💻
</h4>


In [9]:
catalog_element84 = Client.open("https://earth-search.aws.element84.com/v1")
catalog_mpc = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1", modifier=planetary_computer.sign_inplace)

<br><h4 style="color: #d35400; font-family: 'Segoe UI', sans-serif; margin-top: 20px; margin-bottom: 10px; display: flex; align-items: center;">
    📍🌍 Defining Area of Interest (AOI): Latitude and Longitude Coordinates 🗺️📊
</h4>


In [49]:
lat,lon = 25.750,77.905
polygon = Point(lon,lat).buffer(2000*(1/111/1000))
daterange = "2023-05-01/2024-05-01"

<br><h4 style="color: #3498db; font-family: 'Segoe UI', sans-serif; margin-top: 20px; margin-bottom: 10px; display: flex; align-items: center;">
    🌿📡 Downloading Year-Long NDVI Data from Sentinel-2 🌍📊
</h4>


In [50]:
def download_datacube(polygon,daterange):
    """
    Download and Process Datacube for NDVI and LULC.

    Downloads Sentinel-2 data for a given polygon and date range, computes NDVI,
    fills missing values, and aligns it with LULC data.

    Parameters:
    -----------
    polygon : shapely.geometry.Polygon
        Geographical area of interest.
    daterange : str
        Date range in ISO format (e.g., "2023-01-01/2023-12-31").
    """
    min_lon,max_lon = np.array(list(polygon.exterior.coords))[:,0].min(), np.array(list(polygon.exterior.coords))[:,0].max()
    min_lat,max_lat = np.array(list(polygon.exterior.coords))[:,1].min(), np.array(list(polygon.exterior.coords))[:,1].max()
    bbox = min_lon, min_lat, max_lon, max_lat
    query = catalog_element84.search(collections=["sentinel-2-l2a"], datetime=daterange, limit=100, bbox=bbox)
    query=list(query.items())

    ds = load(query,geopolygon=polygon,groupby="solar_day",bands=["red", "green", "blue","nir",'swir16','scl'],chunks={"x":1000,"y":1000,"time":-1}).astype("float32").transpose('y','x','time')
    ds['NDVI'] = (ds.nir-ds.red)/(ds.red+ds.nir+1e-3)
    ds_ndvi = ds.NDVI
    ds_scl = ds.scl
    ds_ndvi.data[((ds_scl.data==8) | (ds_scl.data==9) | (ds_scl.data==10) | (ds_ndvi.data==0))] = np.nan

    for time_index in range(len(ds_ndvi.time.values[:-1])):
        curr_time = ds_ndvi.time.values[time_index]
        nearest_time = ds_ndvi.time.values[time_index-1] if (abs(ds_ndvi.time.values[time_index-1]-curr_time)-abs(ds_ndvi.time.values[time_index+1]-curr_time))<0 else ds_ndvi.time.values[time_index+1]
        ds_ndvi.loc[:,:,curr_time] = xr.where(np.isnan(ds_ndvi.sel(time=curr_time)),ds_ndvi.sel(time=nearest_time),ds_ndvi.sel(time=curr_time))

    times = pd.date_range(daterange.split("/")[0],daterange.split("/")[1],freq='10D')
    ds_ndvi = ds_ndvi.sel(time=times,method='nearest')
    ds_ndvi['time'] = times

    query = catalog_mpc.search(collections=["io-lulc-annual-v02"],datetime = "2023-05-01/2024-05-01", limit=100, bbox=bbox)
    query=list(query.items())
    ds_lulc = load(query,geopolygon=polygon,groupby="solar_day",bands=["data"],chunks={}).astype("float32").transpose('y','x','time')['data'].isel(time=0)
    ds_lulc = ds_lulc.compute()
    ds_lulc = ds_lulc.rio.reproject(ds_ndvi.rio.crs)

    ds_lulc = ds_lulc.sel(x=ds_ndvi.x.values,y=ds_ndvi.y.values,method='nearest')
    ds_lulc['y'], ds_lulc['x'] = ds_ndvi.y.values, ds_ndvi.x.values

    return ds_ndvi, ds_lulc

In [51]:
ds_ndvi, ds_lulc = download_datacube(polygon,daterange)
ds_ndvi

<xarray.DataArray 'NDVI' (y: 400, x: 363, time: 37)> Size: 21MB
dask.array<getitem, shape=(400, 363, 37), dtype=float32, chunksize=(400, 363, 37), chunktype=numpy.ndarray>
Coordinates:
  * y            (y) float64 3kB 2.853e+06 2.853e+06 ... 2.849e+06 2.849e+06
  * x            (x) float64 3kB 7.896e+05 7.896e+05 ... 7.932e+05 7.932e+05
    spatial_ref  int32 4B 32643
  * time         (time) datetime64[ns] 296B 2023-05-01 2023-05-11 ... 2024-04-25

<h4 style="color: #11119c; font-family: 'Segoe UI', sans-serif; margin-top: 20px; margin-bottom: 10px;">
    🌐 Forest Mask Computation: Leveraging Maximum Green Days Algorithm 🌿🖥️
</h4>

In [52]:
%%time
with ProgressBar():
  ds_ndvi = ds_ndvi.compute(scheduler='threads',n_workers=4)


ds_forest = ds_ndvi.map_blocks(get_max_consecutive_green_days_xarray,template=xr.zeros_like(ds_ndvi.isel(time=[-1]),dtype='uint8'))
ds_forest = xr.where(ds_lulc.isin([5,6,7,8,1])&(ds_forest<=36),0,ds_forest)

[########################################] | 100% Completed | 75.37 s
CPU times: user 19.2 s, sys: 22 s, total: 41.1 s
Wall time: 1min 17s


<br><h4 style="color: #34495e; font-family: 'Segoe UI', sans-serif; margin-top: 20px; margin-bottom: 10px;">
    🌍📊 Visualizing Forest Mask: Insights from Maximum Consecutive Green Days 🌲📊
</h4>

In [53]:
ds_forest_4326=ds_forest.rio.write_crs(ds_ndvi.rio.crs,inplace=True).transpose('time', 'y', 'x').rio.reproject("epsg:4326").rename({'y':'Latitude','x':'Longitude'})
ds_forest_4326.data[ds_forest_4326>100]=0
ds_forest_4326 = xr.DataArray(data = ds_forest_4326.isel(time=0).data,
             dims=['Latitude','Longitude'],
             coords={
                 'Latitude':ds_forest_4326.Latitude.values,
                 'Longitude':ds_forest_4326.Longitude.values,
             })
ds_forest_4326.data[:,:] = np.where(ds_forest_4326.data[:,:]>=18,1,0)

In [54]:
ds_lulc_4326=ds_lulc.rio.write_crs(ds_lulc.rio.crs,inplace=True).transpose( 'y', 'x').rio.reproject("epsg:4326").rename({'y':'Latitude','x':'Longitude'})
ds_lulc_4326.data[ds_lulc_4326>100]=0
ds_lulc_4326 = xr.DataArray(data = ds_lulc_4326.data,
             dims=['Latitude','Longitude'],
             coords={
                 'Latitude':ds_lulc_4326.Latitude.values,
                 'Longitude':ds_lulc_4326.Longitude.values,
             })

In [55]:
gpd.GeoDataFrame([{'geometry':polygon}],crs='epsg:4326').hvplot(geo=True,color='red',alpha=0.0,tiles='ESRI',height=500,width=500,crs='epsg:4326',title='ESRI Basemap')+\
ds_forest_4326.hvplot(x='Longitude',y='Latitude',height=500,width=500,shared_axes=True,crs='epsg:4326',title='Forest Mask (Maximum Consecutive Green Days)',cmap='purples')+\
xr.where(ds_lulc_4326.isin([2]),1,0).hvplot(x='Longitude',y='Latitude',height=500,width=500,shared_axes=True,crs='epsg:4326',title='Forest Mask (ESRI Model LULC)',cmap='purples')

:Layout
   .Overlay.I :Overlay
      .WMTS.I     :WMTS   [Longitude,Latitude]
      .Polygons.I :Polygons   [Longitude,Latitude]
   .Image.I   :Image   [Longitude,Latitude]   (value)
   .Image.II  :Image   [Longitude,Latitude]   (value)

<br><h2 style="color: #1111aa; font-family: 'Segoe UI', sans-serif; margin-top: 30px; margin-bottom: 20px;">
    📈🌳 Time Series Analysis: Unveiling Forest Change Dynamics Over the Years 🌳📈
</h2>


In [ ]:
def get_forest_area(polygon,daterange):
    ds_ndvi, ds_lulc = download_datacube(polygon,daterange)
    ds_ndvi= ds_ndvi.chunk({'x':1000,'y':1000,'time':37})
    ds_forest = ds_ndvi.map_blocks(get_max_consecutive_green_days_xarray,template=xr.zeros_like(ds_ndvi.isel(time=[-1]),dtype='uint8'))

    print(f"-----------------Running Computation of {daterange}----------------------")
    with ProgressBar():
        ds_forest = ds_forest.compute(scheduler='processes')
    ds_forest = xr.where(ds_lulc.isin([5,6,7,8,1])&(ds_forest<=36),0,ds_forest)
    ds_forest.data[ds_forest>100]=0
    ds_forest.data[:,:] = np.where(ds_forest.data[:,:]>=18,1,0)
    clear_output(wait=True)
    return (ds_forest.data==1).sum()*100/1000/1000

In [ ]:
dfs = []
for year in range(2018,2025):
    dfs.append({
        'year':year,
        'area': get_forest_area(polygon,f'{year-1}-05-01/{year}-05-01')
    })
df = pd.DataFrame(dfs)

-----------------Running Computation of 2023-05-01/2024-05-01----------------------
[########################################] | 100% Completed | 172.01 s


In [ ]:
df = df.set_index('year')
df.index = pd.to_datetime(df.index.astype('str'))

In [ ]:
import hvplot.pandas

# Define the styled plot
line_plot = df.rolling(window=1).mean().hvplot(
    kind='line',
    grid=True,
    height=450,
    width=1200,
    title='🌳 Forest Cover Over the Years using S2 Consecutive Green Days (in KM²) 🌍\n\n',
    line_width=3,
    color='#2ecc71',  # Forest green color for the line
    xlabel='Year',
    ylabel='Forest Cover (KM²)'
).opts(
    fontscale=1.5  # Adjusts font sizes proportionally
)

scatter_plot = df.rolling(window=1).mean().hvplot.scatter(
    grid=True,
    color='red',
    alpha=0.7,
    size=200,
    marker='o',
    hover_cols='all'
).opts(
    fontscale=1.5  # Adjusts font sizes proportionally
)

# Overlay line and scatter plots
(line_plot * scatter_plot).opts(
    legend_position='top_left',  # Better legend placement
    toolbar='above',  # Place toolbar for interactivity above the plot
    xlabel='Year',
    ylabel='Forest Cover (KM²)'
)


:Overlay
   .Curve.I   :Curve   [year]   (area)
   .Scatter.I :Scatter   [year]   (area)